Author: Gup

Practicing Neural Network Implementation on very simple case of identifying B+ or B- mesons from

$B^{\pm}\rightarrow K^{\pm}K^{+}K^{-}$

based on given kaon charge data

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
import glob

2023-01-25 21:52:38.492745: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
%pip install tensorflow
%pip install keras

  Using cached tensorflow-2.11.0-cp39-cp39-macosx_10_14_x86_64.whl (244.3 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached protobuf-3.19.6-cp39-cp39-macosx_10_9_x86_64.whl (980 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached grpcio-1.51.1-cp39-cp39-macosx_10_10_x86_64.whl (4.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 17.5 MB/s eta 0:00:0000:0100:01
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 26.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.9 MB/s eta 0:00:00a 0:00:01
  Using cached tensorflow_estimator-2.11.0-py2.py3-none-any.whl (439 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB

In [3]:
files = []
eventData = []

list_of_files = glob.glob('**/*.csv', recursive=True)
for i in list_of_files:
    files.append(i)

files

['ignore/B2KKK_MagnetDown.csv', 'ignore/B2KKK_MagnetUp.csv']

In [4]:
column_names = ["B_FlightDistance", "B_VertexChi2", "H1_PX", "H1_PY", "H1_PZ", "H1_ProbK", "H1_ProbPi", "H1_Charge", "H1_IPChi2", 
"H2_PX", "H2_PY", "H2_PZ", "H2_ProbK", "H2_ProbPi", "H2_Charge", "H2_IPChi2", 
"H3_PX", "H3_PY", "H3_PZ", "H3_ProbK", "H3_ProbPi", "H3_Charge", "H3_IPChi2"]

# Read the CSV file and assign the column names from the first ro
df1 = pd.read_csv("ignore/B2KKK_MagnetUp.csv", header = None, names = column_names, on_bad_lines = "skip", engine = "python", comment = "#")

df2 = pd.read_csv("ignore/B2KKK_MagnetDown.csv", header = None, names = column_names, on_bad_lines = "skip", engine = "python", comment = "#")

df = pd.concat([df1, df2], ignore_index=True)
df

,B_FlightDistance,B_VertexChi2,H1_PX,H1_PY,H1_PZ,H1_ProbK,H1_ProbPi,H1_Charge,H1_IPChi2,H2_PX,...,H2_ProbPi,H2_Charge,H2_IPChi2,H3_PX,H3_PY,H3_PZ,H3_ProbK,H3_ProbPi,H3_Charge,H3_IPChi2
0,10.42810,4.05695,-4168.060,-704.9540,24548.8,0.771199,0.051726,-1,1002.66000,-3393.640,...,0.038719,1,714.17500,-8864.96,-7428.640,77446.2,0.934580,0.128720,1,536.5850
1,33.59130,2.22024,1295.910,-61.9569,35569.6,0.932006,0.058690,-1,6672.09000,613.485,...,0.043657,1,737.22000,-341.75,3317.420,24380.2,0.968883,0.129317,1,23332.5000
2,16.53190,11.59340,1493.050,1944.6000,33003.9,0.953512,0.104768,-1,1731.92000,373.614,...,0.039157,1,499.99900,4779.03,-287.830,53908.5,0.844950,0.064803,1,3484.1900
3,4.10280,11.32170,-236.694,843.8090,28207.1,0.655129,0.045339,-1,5.03376,-561.906,...,0.040908,1,6.58870,-1085.33,-3076.760,30036.2,0.912293,0.080841,1,489.0470
4,3.50935,5.73940,123.874,856.2900,51956.6,0.940484,0.078220,-1,7.00589,110.753,...,0.041739,1,2.09042,1640.00,2660.280,19441.3,0.702135,0.028641,1,888.5350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26954,31.09040,3.31873,-868.888,-429.5470,28948.2,0.724375,0.039420,-1,1349.73000,-1417.660,...,0.056467,1,1597.50000,-7141.53,-3904.320,83336.7,0.525309,0.129747,1,3879.8800
26955,6.88156,3.87262,588.582,523.0460,17737.8,0.741060,0.055822,1,214.29600,-484.077,...,0.045518,-1,25.25580,-6479.90,1695.490,68440.2,0.957104,0.092155,-1,434.4930
26956,6.27275,11.17060,392.544,1041.1900,15034.3,0.765920,0.071357,1,233.88300,-1347.500,...,0.025600,-1,1859.50000,-4552.41,6281.470,75866.3,0.923278,0.064446,-1,62.5421
26957,24.39090,3.46987,-889.297,431.4790,14038.3,0.931028,0.055747,-1,7235.70000,-2258.700,...,0.037033,1,5225.58000,-5104.51,-3882.280,39723.6,0.976290,0.164023,1,17515.2000


Calculate the B+ and B- by hand to train the neural net (redudant but it's for the sake of practice)

In [5]:
negativeCharges = 0
positiveCharges = 0

H1_Charge = np.array(df.loc[:,"H1_Charge"], dtype = float)
H2_Charge = np.array(df.loc[:,"H2_Charge"], dtype = float)
H3_Charge = np.array(df.loc[:,"H3_Charge"], dtype = float)

for i in range(len(H1_Charge)):
    if (H1_Charge[i] + H2_Charge[i] + H3_Charge[i] == 1):
        positiveCharges += 1
    if (H1_Charge[i] + H2_Charge[i] + H3_Charge[i] == -1):
        negativeCharges += 1
        
print("$B^{+}$ decays =", positiveCharges)
print("$B^{-}$ decays =", negativeCharges)

A_global = (negativeCharges - positiveCharges) / (negativeCharges + positiveCharges)
print("A_global = ", A_global)

$B^{+}$ decays = 13957
$B^{-}$ decays = 13002
A_global =  -0.03542416261730776


In [7]:
df_norm = (df - df.min()) / (df.max() - df.min())

df_norm


,B_FlightDistance,B_VertexChi2,H1_PX,H1_PY,H1_PZ,H1_ProbK,H1_ProbPi,H1_Charge,H1_IPChi2,H2_PX,...,H2_ProbPi,H2_Charge,H2_IPChi2,H3_PX,H3_PY,H3_PZ,H3_ProbK,H3_ProbPi,H3_Charge,H3_IPChi2
0,0.020551,0.338041,0.307151,0.499763,0.143025,0.563296,0.044597,0.0,0.004840,0.370936,...,0.030132,1.0,0.001067,0.221781,0.171146,0.458512,0.904111,0.131418,1.0,0.005120
1,0.084640,0.184953,0.532719,0.528786,0.217345,0.897405,0.052066,0.0,0.032233,0.572823,...,0.035479,1.0,0.001102,0.513923,0.614383,0.128618,0.975476,0.132084,1.0,0.223059
2,0.037439,0.966194,0.540858,0.619356,0.200043,0.942088,0.101487,0.0,0.008363,0.560738,...,0.030606,1.0,0.000747,0.689443,0.465680,0.312186,0.717641,0.060157,1.0,0.033301
3,0.003049,0.943548,0.469449,0.569669,0.167695,0.322137,0.037746,0.0,0.000019,0.513605,...,0.032502,1.0,0.000008,0.488436,0.350646,0.163780,0.857744,0.078038,1.0,0.004666
4,0.001407,0.478271,0.484334,0.570233,0.327853,0.915020,0.073014,0.0,0.000029,0.547495,...,0.033403,1.0,0.000002,0.581849,0.587279,0.097915,0.420524,0.019841,1.0,0.008485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26954,0.077721,0.276511,0.443350,0.512194,0.172693,0.466010,0.031398,0.0,0.006517,0.470490,...,0.049352,1.0,0.002389,0.280853,0.316512,0.495131,0.052650,0.132563,1.0,0.037084
26955,0.010738,0.322677,0.503519,0.555191,0.097094,0.500676,0.048990,1.0,0.001031,0.517526,...,0.037494,0.0,0.000036,0.303531,0.547485,0.402524,0.950970,0.090652,0.0,0.004144
26956,0.009053,0.930954,0.495426,0.578578,0.078863,0.552328,0.065652,1.0,0.001125,0.474025,...,0.015924,0.0,0.002781,0.369598,0.736640,0.448690,0.880598,0.059759,0.0,0.000588
26957,0.059184,0.289108,0.442507,0.551058,0.072146,0.895373,0.048909,0.0,0.034956,0.428117,...,0.028306,1.0,0.007819,0.350674,0.317421,0.224003,0.990886,0.170778,1.0,0.167443


In [9]:
# len(column_names)

# from sklearn.model_selection import train_test_split

# # Split the data into input variables (x) and output variable (y)
# x = df_norm[:, :-1]  # all rows and all columns except the last one
# y = df_norm[:, -1]  # all rows and the last column

# # Split the data into training and test sets
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


Build the model, input is shape (.80 * 26959, 23, 1)

In [10]:
model = Sequential()
tf.keras.layers.InputLayer(input_shape = 23)
model.add(Dense(6))
model.add(Activation("relu"))
model.add(Dense(1))

loss=tf.keras.losses.BinaryCrossentropy(from_logits = False)
model.compile(loss= "categorical_crossentropy", optimizer= "adam", metrics= ["accuracy"])

2023-01-25 22:06:06.593405: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
#model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))